# Setup the workspace

In [1]:
import os
from google.colab import drive

In [2]:
os.chdir("/content/drive/MyDrive/AICUP2020/")

# Prepare Environments

In [3]:
!pip install kashgari>=2.0.0 &> /dev/null
from datetime import datetime
from configparser import ConfigParser
import numpy as np
import pandas as pd
import tensorflow as tf

import kashgari
from kashgari.embeddings import BertEmbedding
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from kashgari.callbacks import EvalCallBack
from model_acceptance_callback import NERAcceptanceCallBack

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from utils import load_test_file, format_result, split_chunks, save_model
from data_generator import read_data, generate_dataset

In [5]:
# Define const
DATE = datetime.now().strftime("%Y%m%d")
MODEL_SAVE_DIR = 'model'

# Get config
config = ConfigParser()
config.read('config.ini')
LSTM_UNITS = int(config['MODEL']['lstm_units'])
DROPOUT = float(config['MODEL']['dropout'])
EPOCHS = int(config['TRAIN']['epochs'])
BATCH_SIZE = int(config['TRAIN']['batch_size'])
PREDICT_BATCH_SIZE = int(config['PREDICT']['batch_size'])

# Read Training Data

In [6]:
# x is a list of characters, e.g ['你', '好', '嗎', ...]
# y is a list of labels of each characters, e.g ['O', 'O', 'O', ...]
training_data = read_data(['data/TRAIN'], end_flag='')
train_x = list(map(lambda x: [wl[0] for wl in x], training_data))
train_y = list(map(lambda x: [wl[1] for wl in x], training_data))

validation_data = read_data(['data/TEST'], end_flag='')
valid_x = list(map(lambda x: [wl[0] for wl in x], validation_data))
valid_y = list(map(lambda x: [wl[1] for wl in x], validation_data))

In [7]:
train_x_chunks = []
for article in train_x:
  article_chunks = split_chunks(article)
  train_x_chunks.extend(article_chunks)

train_y_chunks = []
for article in train_y:
  article_chunks = split_chunks(article)
  train_y_chunks.extend(article_chunks)

valid_x_chunks = []
for article in valid_x:
  article_chunks = split_chunks(article)
  valid_x_chunks.extend(article_chunks)

valid_y_chunks = []
for article in valid_y:
  article_chunks = split_chunks(article)
  valid_y_chunks.extend(article_chunks)

# Build Model

In [8]:
# build model
hyper_parameters = {
  'layer_blstm': {
    'units': LSTM_UNITS,
    'return_sequences': True
  },
  'layer_dropout': {
    'rate': DROPOUT
  },
  'layer_time_distributed': {

  },
  'layer_activation': {
    'activation': 'softmax'
  }
}
bert_embed = BertEmbedding('chinese_L-12_H-768_A-12/')
model = BiLSTM_CRF_Model(bert_embed, hyper_parameters=hyper_parameters)  # 他要求最大的sequence_length只能是512，對應到bert的max_position

2020-12-10 11:36:26,277 [DEBUG] kashgari - ------------------------------------------------
2020-12-10 11:36:26,278 [DEBUG] kashgari - Loaded transformer model's vocab
2020-12-10 11:36:26,281 [DEBUG] kashgari - config_path       : chinese_L-12_H-768_A-12/bert_config.json
2020-12-10 11:36:26,282 [DEBUG] kashgari - vocab_path      : chinese_L-12_H-768_A-12/vocab.txt
2020-12-10 11:36:26,283 [DEBUG] kashgari - checkpoint_path : chinese_L-12_H-768_A-12/bert_model.ckpt
2020-12-10 11:36:26,284 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused32]', '[unused33]', '[unused

In [8]:
# load model
model_dir = '20201209_koi'
model = BiLSTM_CRF_Model.load_model(f'model/{model_dir}/')
model.crf_layer = model.layer_crf
model.compile_model()

2020-12-10 10:47:09,832 [DEBUG] kashgari - ------------------------------------------------
2020-12-10 10:47:09,833 [DEBUG] kashgari - Loaded transformer model's vocab
2020-12-10 10:47:09,833 [DEBUG] kashgari - config_path       : chinese_L-12_H-768_A-12/bert_config.json
2020-12-10 10:47:09,836 [DEBUG] kashgari - vocab_path      : chinese_L-12_H-768_A-12/vocab.txt
2020-12-10 10:47:09,838 [DEBUG] kashgari - checkpoint_path : chinese_L-12_H-768_A-12/bert_model.ckpt
2020-12-10 10:47:09,840 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused32]', '[unused33]', '[unused

In [ ]:
# model.compile_model()
ner_cb = NERAcceptanceCallBack(f'{MODEL_SAVE_DIR}/{DATE}_koi', model, valid_x_chunks, valid_y_chunks)
history = model.fit(train_x_chunks,
           train_y_chunks,
           x_validate=valid_x_chunks,
           y_validate=valid_y_chunks,
           epochs=80,
           batch_size=16, 
           callbacks=[ner_cb])

2020-12-09 14:42:21,524 [DEBUG] kashgari - fit input shape: (2, 16, 511)
2020-12-09 14:42:21,525 [DEBUG] kashgari - fit input shape: (16, 511)


Epoch 1/80
83/83 [==============================] - ETA: 0s - loss: 6.5563 - accuracy: 0.9927

2020-12-09 14:47:22,125 [WARNING] kashgari - Sequence length is None, will use the max embedding seq length, which is 512
2020-12-09 14:47:22,151 [DEBUG] kashgari - predict seq_length: None, input: (2, 154, 512)


10/10 [==============================] - 15s 1s/step


2020-12-09 14:47:44,334 [DEBUG] kashgari - predict output: (154, 512)
2020-12-09 14:47:44,335 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 ...
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]]
2020-12-09 14:47:44,905 [INFO] kashgari - Successfully recorded this epoch into model/20201209_koi/history/history_1.csv
2020-12-09 14:47:44,907 [INFO] kashgari - 
Epoch 00001: f1-score improved from -inf to 0.77259, saving model to model/20201209_koi


...


            precision    recall  f1-score   support

      name     0.8438    0.9310    0.8852        29
      time     0.9087    0.9645    0.9358       423
  med_exam     0.9459    0.9589    0.9524        73
     money     1.0000    1.0000    1.0000        15
  location     1.0000    0.8727    0.9320        55
   contact     1.0000    0.5333    0.6957        15
    family     1.0000    0.7500    0.8571        12
profession     1.0000    1.0000    1.0000         1
 education     1.0000    1.0000    1.0000         1
        ID     1.0000    1.0000    1.0000         1

 micro avg     0.9216    0.9408    0.9311       625
 macro avg     0.9246    0.9408    0.9296       625


epoch: 78 precision: 0.924635, recall: 0.940800, f1-score: 0.929621
83/83 [==============================] - 273s 3s/step - loss: 0.4173 - accuracy: 0.9992 - val_loss: 592.0454 - val_accuracy: 0.0137
Epoch 80/80
83/83 [==============================] - ETA: 0s - loss: 0.5577 - accuracy: 0.9989

2020-12-09 20:59:21,098 [DEBUG] kashgari - predict seq_length: None, input: (2, 154, 512)


10/10 [==============================] - 14s 1s/step


2020-12-09 20:59:37,025 [DEBUG] kashgari - predict output: (154, 512)
2020-12-09 20:59:37,026 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 ...
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]]
2020-12-09 20:59:37,573 [INFO] kashgari - Successfully recorded this epoch into model/20201209_koi/history/history_1.csv
2020-12-09 20:59:37,574 [INFO] kashgari - 
Epoch 00080: f1-score did not improve from 0.93329



            precision    recall  f1-score   support

      name     0.8438    0.9310    0.8852        29
      time     0.9269    0.9598    0.9431       423
  med_exam     0.9577    0.9315    0.9444        73
     money     0.9375    1.0000    0.9677        15
  location     1.0000    0.8727    0.9320        55
   contact     1.0000    0.4667    0.6364        15
    family     1.0000    0.6667    0.8000        12
profession     1.0000    1.0000    1.0000         1
 education     1.0000    1.0000    1.0000         1
        ID     1.0000    1.0000    1.0000         1

 micro avg     0.9342    0.9312    0.9327       625
 macro avg     0.9369    0.9312    0.9303       625


epoch: 79 precision: 0.936868, recall: 0.931200, f1-score: 0.930347
83/83 [==============================] - 273s 3s/step - loss: 0.5577 - accuracy: 0.9989 - val_loss: 595.1830 - val_accuracy: 0.0143


# Check Performation

In [6]:
# perf = pd.DataFrame(data=history.history)
perf1 = pd.read_csv('model/20201209_koi/history/history_0.csv')
perf2 = pd.read_csv('model/20201209_koi/history/history_1.csv')
perf = pd.concat([perf1, perf2], ignore_index=True)
perf

,loss,accuracy,val_loss,val_accuracy,precision,recall,f1-score
0,90.391586,0.958680,1164.874756,0.975973,0.027724,0.0272,0.027460
1,32.981873,0.978497,1184.099121,0.982153,0.263793,0.2944,0.275243
2,23.283619,0.982453,1164.145752,0.984537,0.393113,0.4464,0.414983
3,18.903397,0.984735,1153.581665,0.985872,0.503248,0.4208,0.452370
4,16.318130,0.986312,1141.170898,0.988427,0.577034,0.6288,0.598528
...,...,...,...,...,...,...,...
91,0.477505,0.999207,603.543152,0.014591,0.927754,0.9344,0.927621
92,0.430426,0.999266,605.662354,0.014508,0.926512,0.9264,0.922161
93,0.492822,0.999108,598.514099,0.013558,0.926605,0.9360,0.927220
94,0.417326,0.999233,592.045410,0.013733,0.924635,0.9408,0.929621


In [13]:
fig = make_subplots(rows=2, cols=2,
           specs=[[{}, {}], [{'colspan': 2}, None]],
           subplot_titles=('Loss and Validate loss', 'Accuracy and Validate accuracy', 'F1 Score'))
fig.add_trace(go.Scatter(y=perf['loss'], name='loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=perf['val_loss'], name='validate loss'), row=1, col=1)

fig.add_trace(go.Scatter(y=perf['accuracy'], name='accuracy'), row=1, col=2)
fig.add_trace(go.Scatter(y=perf['val_accuracy'], name='validate accuracy'), row=1, col=2)

for prop in perf.columns[4:]:
  fig.add_trace(go.Scatter(y=perf[prop], name=prop), row=2, col=1)

fig.update_layout(height=800, width=1000)

fig.show()

'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plotly.com/~SharpKoi/20201209.embed" height="525" width="100%"></iframe>'

In [14]:
!pip install chart_studio > /dev/null
import chart_studio.tools as tls
import chart_studio.plotly as cplt

usr = 'SharpKoi'
api_key = 'XwSSNCqGzY0TWCDthlkC'
tls.set_credentials_file(username=usr, api_key=api_key)
cplt.plot(fig, file_name='20201209_e96_bs16', auto_open=False)

'https://plotly.com/~SharpKoi/1/'

In [ ]:
model.evaluate(valid_x_chunks, valid_y_chunks)

2020-12-10 02:19:05,008 [WARNING] kashgari - Sequence length is None, will use the max embedding seq length, which is 512
2020-12-10 02:19:05,039 [DEBUG] kashgari - predict seq_length: None, input: (2, 154, 512)


5/5 [==============================] - 6s 1s/step


2020-12-10 02:19:23,988 [DEBUG] kashgari - predict output: (154, 512)
2020-12-10 02:19:23,989 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 ...
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]]



            precision    recall  f1-score   support

      time     0.9292    0.9622    0.9454       423
      name     0.8182    0.9310    0.8710        29
  location     1.0000    0.8727    0.9320        55
  med_exam     0.9589    0.9589    0.9589        73
    family     0.9000    0.7500    0.8182        12
     money     0.9375    1.0000    0.9677        15
   contact     1.0000    0.4667    0.6364        15
        ID     1.0000    1.0000    1.0000         1
profession     1.0000    1.0000    1.0000         1
 education     1.0000    1.0000    1.0000         1

 micro avg     0.9331    0.9376    0.9354       625
 macro avg     0.9354    0.9376    0.9333       625



{'detail': {'ID': {'f1-score': 1.0,
   'precision': 1.0,
   'recall': 1.0,
   'support': 1},
  'contact': {'f1-score': 0.6363636363636364,
   'precision': 1.0,
   'recall': 0.4666666666666667,
   'support': 15},
  'education': {'f1-score': 1.0,
   'precision': 1.0,
   'recall': 1.0,
   'support': 1},
  'family': {'f1-score': 0.8181818181818182,
   'precision': 0.9,
   'recall': 0.75,
   'support': 12},
  'location': {'f1-score': 0.9320388349514563,
   'precision': 1.0,
   'recall': 0.8727272727272727,
   'support': 55},
  'med_exam': {'f1-score': 0.958904109589041,
   'precision': 0.958904109589041,
   'recall': 0.958904109589041,
   'support': 73},
  'money': {'f1-score': 0.967741935483871,
   'precision': 0.9375,
   'recall': 1.0,
   'support': 15},
  'name': {'f1-score': 0.8709677419354839,
   'precision': 0.8181818181818182,
   'recall': 0.9310344827586207,
   'support': 29},
  'profession': {'f1-score': 1.0,
   'precision': 1.0,
   'recall': 1.0,
   'support': 1},
  'time': {'f1-s

# Prediction

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# load model
save_date = '20201130_mike'
model = BiLSTM_CRF_Model.load_model(f'model/{save_date}/')
model.crf_layer = model.layer_crf

2020-12-02 13:11:51,218 [DEBUG] kashgari - ------------------------------------------------
2020-12-02 13:11:51,219 [DEBUG] kashgari - Loaded transformer model's vocab
2020-12-02 13:11:51,223 [DEBUG] kashgari - config_path       : chinese_L-12_H-768_A-12/bert_config.json
2020-12-02 13:11:51,224 [DEBUG] kashgari - vocab_path      : chinese_L-12_H-768_A-12/vocab.txt
2020-12-02 13:11:51,225 [DEBUG] kashgari - checkpoint_path : chinese_L-12_H-768_A-12/bert_model.ckpt
2020-12-02 13:11:51,227 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused32]', '[unused33]', '[unused

In [ ]:
for_predict = load_test_file('./data/development_2.txt')

In [ ]:
def predict(model: tf.keras.Model, data):
    output_df = pd.DataFrame()
    for article_id in range(len(data)):
        text = list(data[article_id])
        batched_text = split_chunks(text)
        batched_labels = model.predict(batched_text)
        text = [char for batch in batched_text for char in batch]
        labels = [l for batch in batched_labels for l in batch]

        entities_result = format_result(text, labels)
        df = pd.DataFrame(entities_result)
        df.insert(0, 'article_id', [article_id for _ in range(len(entities_result))])
        output_df = pd.concat([output_df, df], ignore_index=True)
    
    return output_df

In [ ]:
ans = predict(model, for_predict)
ans[['article_id', 'start_position', 'end_position']] = ans[['article_id', 'start_position', 'end_position']].astype(int)

2020-12-10 02:20:20,403 [DEBUG] kashgari - predict seq_length: None, input: (2, 7, 512)


1/1 [==============================] - 0s 4ms/step


2020-12-10 02:20:21,106 [DEBUG] kashgari - predict output: (7, 512)
2020-12-10 02:20:21,107 [DEBUG] kashgari - predict output argmax: [[ 0  1  1 ...  1  1  1]
 [ 0  1  1 ...  1  1  1]
 [ 0  1  1 ...  1  1  1]
 ...
 [ 0  1  1 ...  1  1  1]
 [ 0 10  7 ...  1  1  1]
 [ 0  1  3 ...  1  1  1]]
2020-12-10 02:20:21,132 [DEBUG] kashgari - predict seq_length: None, input: (2, 2, 512)


1/1 [==============================] - 0s 4ms/step


2020-12-10 02:20:21,566 [DEBUG] kashgari - predict output: (2, 512)
2020-12-10 02:20:21,567 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]]
2020-12-10 02:20:21,584 [DEBUG] kashgari - predict seq_length: None, input: (2, 3, 512)


...

2020-12-10 02:21:03,363 [DEBUG] kashgari - predict output: (4, 512)
2020-12-10 02:21:03,364 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]]
2020-12-10 02:21:03,382 [DEBUG] kashgari - predict seq_length: None, input: (2, 2, 512)


1/1 [==============================] - 0s 4ms/step


2020-12-10 02:21:03,817 [DEBUG] kashgari - predict output: (2, 512)
2020-12-10 02:21:03,819 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]]
2020-12-10 02:21:03,839 [DEBUG] kashgari - predict seq_length: None, input: (2, 4, 512)


1/1 [==============================] - 0s 4ms/step


2020-12-10 02:21:04,353 [DEBUG] kashgari - predict output: (4, 512)
2020-12-10 02:21:04,354 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]]


In [ ]:
ans

,article_id,start_position,end_position,entity_text,entity_type
0,0,273,276,李醫生,name
1,0,320,322,25,med_exam
2,0,326,328,25,med_exam
3,0,400,403,11點,time
4,0,409,411,4點,time
...,...,...,...,...,...
1405,69,1565,1569,24小時,time
1406,69,1597,1601,9月8號,time
1407,69,1634,1636,7點,time
1408,69,1639,1641,4點,time


In [ ]:
ans.to_csv(f'./output/output_1209_koi.tsv', sep='\t', index=False)